## Importing Libraries

In [1]:
import ast
import ipywidgets as widgets
import json
import pandas as pd
import torch

from IPython.display import display, clear_output
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Metadata

In [3]:
df_metadata_ori = pd.read_csv("data/movies_metadata.csv", low_memory=False)
df_metadata_ori.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30/10/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15/12/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22/12/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22/12/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10/2/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df_metadata_ori.shape

(45466, 24)

In [5]:
df_metadata_ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Checking if there are missing values.

In [6]:
df_metadata_ori.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

## Data Preprocessing

In [7]:
df_metadata = df_metadata_ori.copy()

### Columns that are needed to build our recommendation system

In [8]:
df_metadata = df_metadata[
    [
        "id",
        "title",
        "genres",
        "original_language",
        "overview",
        "tagline",
        "production_countries",
        "release_date",
        "status",
        "vote_average",
        "vote_count",
        "runtime",
    ]
]

### Check if duplicates titles have same release date 

In [9]:
df_metadata[["title", "release_date"]].duplicated().sum()

32

### Number of movies with no overviews

In [10]:
df_metadata[df_metadata.overview.isnull()].shape[0]

954

### Number of movies that have not yet been released

In [11]:
df_metadata[df_metadata.status != "Released"].shape[0]

452

**TODO:**  

We will remove movies which:
- have same titles and release date.
- have no overviews.
- have not yet been released.

### Genres and Production Countries

In [12]:
df_metadata["genres"][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [13]:
df_metadata["production_countries"][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

**TODO:**  

We need to extract the names from the data.

### Extract the movie feature names from the data

In [14]:
def extract_names(data: str) -> str:
    """
    Extract the names from the data.

    :param data: A string representing a list of objects. Each object should have a 'name' key.
    :return: A string containing the names of the features, separated by commas and spaces.

    **Example:**
    >>> extract_names("[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")
    'Animation, Comedy, Family'
    """
    if data:
        try:
            # Convert the input string to a JSON-formatted string
            json_str = json.dumps(ast.literal_eval(data))
            # Load the JSON-formatted string into a Python object
            python_obj = json.loads(json_str)
            # Extract the names from the Python object
            data_names = [data["name"] for data in python_obj]
            # Join the names into a single string separated by spaces
            data_names_str = ", ".join(data_names)

            return data_names_str

        except TypeError:
            return ""
    else:
        # If the input is empty, return an empty string
        return ""

In [15]:
def clean_movies_data_set(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the movies dataset by removing duplicates, null values and non-released movies,
    and extracting the genre names and production countries from the data.

    :param df: The movie dataset to be cleaned.
    :return: A cleaned pandas DataFrame.
    """
    print(f"The number of movies in the original data set is: {df.shape[0]}")

    # Removes duplicates titles that have same release date
    df.drop_duplicates(subset=["title", "release_date"], inplace=True)

    # Removes movies that have no overview or have not yet been released
    index_drop = df[(df.overview.isnull()) | (df.status != "Released")].index
    df.drop(index_drop, inplace=True)

    # Fills the rows with empty production_countries to NaN
    df.loc[df.production_countries == "[]", "production_countries"] = pd.NA

    # Replaces all the null values with empty string
    df.fillna("", inplace=True)

    # Extracts the genre names and production countries from the data
    df["genres"] = df["genres"].apply(extract_names)
    df["production_countries"] = df["production_countries"].apply(extract_names)

    print(f"The number of movies in the cleaned data set is: {df.shape[0]}")

    return df

In [16]:
df_metadata = clean_movies_data_set(df_metadata)

The number of movies in the original data set is: 45466
The number of movies in the cleaned data set is: 44065


In [17]:
df_metadata.head()

,id,title,genres,original_language,overview,tagline,production_countries,release_date,status,vote_average,vote_count,runtime
0,862,Toy Story,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",,United States of America,30/10/1995,Released,7.7,5415.0,81.0
1,8844,Jumanji,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,United States of America,15/12/1995,Released,6.9,2413.0,104.0
2,15602,Grumpier Old Men,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,United States of America,22/12/1995,Released,6.5,92.0,101.0
3,31357,Waiting to Exhale,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,United States of America,22/12/1995,Released,6.1,34.0,127.0
4,11862,Father of the Bride Part II,Comedy,en,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,United States of America,10/2/1995,Released,5.7,173.0,106.0


# Keywords

In [18]:
df_keywords = pd.read_csv("data/keywords.csv")

In [19]:
df_keywords = df_keywords.drop_duplicates()

In [20]:
df_keywords.shape

(45432, 2)

In [21]:
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [22]:
df_keywords.keywords[0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

### Extract keywords

In [23]:
df_keywords["keywords"] = df_keywords["keywords"].apply(extract_names)

In [24]:
df_keywords.keywords[0]

'jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life'

In [25]:
df_keywords.head()

,id,keywords
0,862,"jealousy, toy, boy, friendship, friends, rival..."
1,8844,"board game, disappearance, based on children's..."
2,15602,"fishing, best friend, duringcreditsstinger, ol..."
3,31357,"based on novel, interracial relationship, sing..."
4,11862,"baby, midlife crisis, confidence, aging, daugh..."


## Merge two data frames

In [26]:
df_metadata["id"] = df_metadata["id"].astype(str)
df_keywords["id"] = df_keywords["id"].astype(str)
df_merge = pd.merge(df_keywords, df_metadata, on="id")

In [27]:
df_merge.head()

,id,keywords,title,genres,original_language,overview,tagline,production_countries,release_date,status,vote_average,vote_count,runtime
0,862,"jealousy, toy, boy, friendship, friends, rival...",Toy Story,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",,United States of America,30/10/1995,Released,7.7,5415.0,81.0
1,8844,"board game, disappearance, based on children's...",Jumanji,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,United States of America,15/12/1995,Released,6.9,2413.0,104.0
2,15602,"fishing, best friend, duringcreditsstinger, ol...",Grumpier Old Men,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,United States of America,22/12/1995,Released,6.5,92.0,101.0
3,31357,"based on novel, interracial relationship, sing...",Waiting to Exhale,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,United States of America,22/12/1995,Released,6.1,34.0,127.0
4,11862,"baby, midlife crisis, confidence, aging, daugh...",Father of the Bride Part II,Comedy,en,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,United States of America,10/2/1995,Released,5.7,173.0,106.0


In [28]:
df_merge.shape

(44064, 13)

In [29]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44064 entries, 0 to 44063
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    44064 non-null  object 
 1   keywords              44064 non-null  object 
 2   title                 44064 non-null  object 
 3   genres                44064 non-null  object 
 4   original_language     44064 non-null  object 
 5   overview              44064 non-null  object 
 6   tagline               44064 non-null  object 
 7   production_countries  44064 non-null  object 
 8   release_date          44064 non-null  object 
 9   status                44064 non-null  object 
 10  vote_average          44064 non-null  float64
 11  vote_count            44064 non-null  float64
 12  runtime               44064 non-null  float64
dtypes: float64(3), object(10)
memory usage: 4.4+ MB


# Content-Based Filtering

### Create a new column named 'soup', which is a string contains all the data that we want to feed to the model.  
- Soup: genres, original language, overview, tagline, keywords,  production countries

In [30]:
def create_soup(movie: pd.Series) -> str:
    """
    Concatenates several movies features into a single string to create a soup of text.

    :param movie: A movie containing features to concatenate.
    :return: A string containing the concatenated movie features.
    """
    return (
        movie.genres
        + " "
        + movie.original_language
        + " "
        + movie.overview
        + " "
        + movie.tagline
        + " "
        + movie.keywords
        + " "
        + movie.production_countries
    ).lower()

In [31]:
df_merge["soup"] = df_merge.apply(create_soup, axis=1)

In [32]:
df_merge.soup[0]

"animation, comedy, family en led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences.  jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life united states of america"

In [33]:
df_merge.head()

,id,keywords,title,genres,original_language,overview,tagline,production_countries,release_date,status,vote_average,vote_count,runtime,soup
0,862,"jealousy, toy, boy, friendship, friends, rival...",Toy Story,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",,United States of America,30/10/1995,Released,7.7,5415.0,81.0,"animation, comedy, family en led by woody, and..."
1,8844,"board game, disappearance, based on children's...",Jumanji,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,United States of America,15/12/1995,Released,6.9,2413.0,104.0,"adventure, fantasy, family en when siblings ju..."
2,15602,"fishing, best friend, duringcreditsstinger, ol...",Grumpier Old Men,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,United States of America,22/12/1995,Released,6.5,92.0,101.0,"romance, comedy en a family wedding reignites ..."
3,31357,"based on novel, interracial relationship, sing...",Waiting to Exhale,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,United States of America,22/12/1995,Released,6.1,34.0,127.0,"comedy, drama, romance en cheated on, mistreat..."
4,11862,"baby, midlife crisis, confidence, aging, daugh...",Father of the Bride Part II,Comedy,en,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,United States of America,10/2/1995,Released,5.7,173.0,106.0,comedy en just when george banks has recovered...


In [34]:
# Initialize the model with the 'all-MiniLM-L6-v2' pre-trained model and load it to GPU if available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device)

In [35]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [36]:
# Encode a list of sentences in the 'soup' column using the pre-trained model
sentence_embeddings = model.encode(df_merge["soup"].tolist())

In [37]:
# Compute similarity
cos_sim = cosine_similarity(sentence_embeddings)

In [38]:
def lowercase_title(title: str) -> str:
    """
    Creates a lowercase version of movie title.

    :param title: The movie title.
    :return: A lowercase version of the movie title.
    """
    return title.lower()


# Construct a reverse map of movie titles to indices
movie_indices = pd.Series(
    df_merge.index, index=df_merge["title"].apply(lowercase_title)
).drop_duplicates()

In [39]:
movie_indices.head()

title
toy story                      0
jumanji                        1
grumpier old men               2
waiting to exhale              3
father of the bride part ii    4
dtype: int64

### IMDB's weighted rating

A movie with an average rating of 9 based on and only 2 votes cannot be considered better than a movie with a lower average rating of 8 but has 1000 votes. So we will be using IMDB's weighted rating to determine the quality of a movie.

In [40]:
def weighted_rating(
    movie: pd.Series, min_vote_counts: int, mean_vote_average: float
) -> float:
    """
    Calculate the weighted rating for a movie based on its vote count, vote average,
    and the minimum vote counts and mean vote average across the dataset.

    :param movie: A DataFrame row representing a movie.
    :param min_votes: The mnimum votes required to be listed in the chart.
    :param mean_vote_average: The mean vote average across the whole dataset.
    :return: The weighted rating for the movie.
    """
    vote_count = movie["vote_count"]
    vote_average = movie["vote_average"]
    return (vote_count / (vote_count + min_vote_counts) * vote_average) + (
        min_vote_counts / (min_vote_counts + vote_count) * mean_vote_average
    )

In [41]:
def get_recommended_movie_indices(title: str) -> list[int]:
    """
    Get the indices of the top 50 most similar movies based on the cosine similarity of their metadata and keywords, 
    regardless of the quality of the movie.

    :param title: The title of the movie to find similar movies for.
    :param cosine_sim: A matrix of pairwise cosine similarity scores between all movies, defaults to `cos_sim`.
    :return: A list of recommended movie indices.
    """

    try:
        # Get the index of the movie that matches the title
        movie_index = movie_indices[title.lower()]

        # If there are multiple movies with the same title, pick the first one.
        if isinstance(movie_index, pd.Series):
            movie_index = movie_index[0]

    except KeyError:
        return f"Movie '{title}' not found. Please enter a valid movie title."

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[movie_index]))

    # Sort the movies based on the similarity scores and get the first 50 movie recommendations
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:51]

    # Get the movie indices
    recommended_movie_indices = [sim_score[0] for sim_score in sim_scores]

    return recommended_movie_indices

In [42]:
def filter_out_bad_movies(recommended_movie_indices: list[int]) -> None:
    """
    Print the details of the recommended movies after filtering out bad movies based on IMDB's weighted rating.

    :param recommended_movie_indices: The indices of recommended movies.
    """
    movie_details = [
        "id",
        "title",
        "genres",
        "original_language",
        "production_countries",
        "release_date",
        "overview",
        "vote_count",
        "vote_average",
        "runtime",
    ]

    movies = df_merge.loc[recommended_movie_indices, movie_details]

    # Get the vote counts and vote averages for the movies
    vote_counts = movies[movies["vote_count"].notnull()]["vote_count"]
    vote_averages = movies[movies["vote_average"].notnull()]["vote_average"]

    # Calculate the minimum votes required to be listed in the chart and the mean vote average
    min_vote_counts = vote_counts.quantile(0.6)
    mean_vote_average = vote_averages.mean()

    # Filter the movies based on the conditions for vote count and vote average
    qualified_movies = movies[
        (movies["vote_count"] >= min_vote_counts)
        & (movies["vote_count"].notnull())
        & (movies["vote_average"].notnull())
    ]

    # Apply the weighted_rating function to calculate the weighted ratings for each movie
    qualified_movies["weighted_rating"] = qualified_movies.apply(
        lambda movie: weighted_rating(movie, min_vote_counts, mean_vote_average), axis=1
    )
    qualified_movies = qualified_movies.sort_values("weighted_rating", ascending=False)
    return qualified_movies

In [43]:
def get_movie_recommendations_cbf(title: str, num_recommendations: int = 5) -> None:
    """
    Get movie recommendations based on a given title using content-based filtering.

    :param title: The title of the movie to find similar movies for.
    :param num_recommendations: The number of recommended movies, defaults to 5.
    """

    recommended_movie_indices = get_recommended_movie_indices(title)
    recommended_movies = filter_out_bad_movies(recommended_movie_indices).head(
        num_recommendations
    )

    for _, movie in recommended_movies.iterrows():
        # Print the movie details
        print(f"Title: {movie['title']}")
        print(f"Overview: {movie['overview']}")
        print(f"Genres: {movie['genres']}")
        print(f"Original Language: {movie['original_language']}")
        print(f"Runtime: {int(movie['runtime'])} mins")
        print(f"Production Countries: {movie['production_countries']}")
        print(f"Release Date: {movie['release_date']}")
        print(
            f"Rating: {movie['vote_average']} out of 10 ({int(movie['vote_count'])} ratings)"
        )
        print("")

In [ ]:
dropdown = widgets.Dropdown(options=df_merge["title"].unique())
search_box = widgets.Text(placeholder="Search movie title...")
button = widgets.Button(description="Get Movie Recommendations")
button.layout.width = "200px"


def on_search_box_value_change(change):
    """
    Event listener for the search box widget. Updates the dropdown options based on the search query.
    """

    search_value = change.new.lower()
    options = df_merge[df_merge["title"].str.lower().str.contains(search_value)][
        "title"
    ].unique()
    dropdown.options = options if len(options) > 0 else ["Movie not found"]
    dropdown.label = options[0] if len(options) > 0 else "Movie not found"


def on_button_click(button):
    """
    Event listener for the button widget. Displays the recommended movies.
    """

    clear_output()
    display(search_box)
    display(dropdown)
    display(button)
    title = dropdown.value

    if title != "Movie not found":
        search_box.value = dropdown.value
        get_movie_recommendations_cbf(title)

    else:
        print("Movie Not Found")


# Attach event listeners to the widgets
search_box.observe(on_search_box_value_change, names="value")
button.on_click(on_button_click)

# Display the widgets
display(search_box)
display(dropdown)
display(button)

Text(value='', placeholder='Search movie title...')

Dropdown(index=2024, options=('Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the …

Button(description='Get Movie Recommendations', layout=Layout(width='200px'), style=ButtonStyle())

Title: Superbad
Overview: High school best buddies are facing separation anxiety as they prepare to go off to college. While attempting to score alcohol for a party with help from a fake ID-toting friend, the guys' evening takes a turn into chaotic territory.
Genres: Comedy
Original Language: en
Runtime: 113 mins
Production Countries: United States of America
Release Date: 17/8/2007
Rating: 7.0 out of 10 (2152 ratings)

Title: The Kings of Summer
Overview: Joe Toy, on the verge of adolescence, finds himself increasingly frustrated by his single father, Frank's attempts to manage his life. Declaring his freedom once and for all, he escapes to a clearing in the woods with his best friend, Patrick, and a strange kid named Biaggio. He announces that they are going to build a house there, free from responsibility and parents. Once their makeshift abode is finished, the three young men find themselves masters of their own destiny, alone in the woods.
Genres: Comedy, Drama
Original Language: 

# Collaborative Filtering

In [83]:
df_ratings = pd.read_csv("data/ratings_small.csv")

In [84]:
df_ratings.shape

(100004, 4)

In [85]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [86]:
reader = Reader()
data = Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader)

In [87]:
svd = SVD()
cross_validate(svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8851  0.8966  0.9019  0.9054  0.8999  0.8978  0.0070  
MAE (testset)     0.6841  0.6905  0.6907  0.6984  0.6922  0.6912  0.0046  
Fit time          0.78    0.69    0.70    0.67    0.70    0.71    0.04    
Test time         0.08    0.07    0.07    0.07    0.07    0.07    0.00    


{'test_rmse': array([0.88508951, 0.89661886, 0.90187904, 0.90544375, 0.89992305]),
 'test_mae': array([0.68407235, 0.69047089, 0.69066488, 0.69837561, 0.69223796]),
 'fit_time': (0.7807812690734863,
  0.6947872638702393,
  0.6961324214935303,
  0.6745295524597168,
  0.6987829208374023),
 'test_time': (0.07694053649902344,
  0.07264065742492676,
  0.070220947265625,
  0.06841564178466797,
  0.06806826591491699)}

In [88]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [157]:
df_ratings[df_ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [156]:
# Predict the rating of User 1 for the movie with the movie ID 12.
user_id = 1
movie_id = 12

predicted_rating = svd.predict(user_id, movie_id)
predicted_rating.est

2.316828416951106

# Hybrid Recommender

In [52]:
df_ids = pd.read_csv("data/links.csv")[["movieId", "tmdbId"]]
df_ids.head()

,movieId,tmdbId
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0


In [53]:
df_ids.dropna(inplace=True)

In [54]:
df_ids.shape

(45624, 2)

In [55]:
df_ids.columns = ["movieId", "id"]
df_ids["id"] = df_ids["id"].astype(int)
df_ids["id"] = df_ids["id"].astype(str)

In [56]:
df_merge_ids = df_merge.merge(df_ids, on="id")

In [57]:
df_merge_ids.head()

,id,keywords,title,genres,original_language,overview,tagline,production_countries,release_date,status,vote_average,vote_count,runtime,soup,movieId
0,862,"jealousy, toy, boy, friendship, friends, rival...",Toy Story,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",,United States of America,30/10/1995,Released,7.7,5415.0,81.0,"animation, comedy, family en led by woody, and...",1
1,8844,"board game, disappearance, based on children's...",Jumanji,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,United States of America,15/12/1995,Released,6.9,2413.0,104.0,"adventure, fantasy, family en when siblings ju...",2
2,15602,"fishing, best friend, duringcreditsstinger, ol...",Grumpier Old Men,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,United States of America,22/12/1995,Released,6.5,92.0,101.0,"romance, comedy en a family wedding reignites ...",3
3,31357,"based on novel, interracial relationship, sing...",Waiting to Exhale,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,United States of America,22/12/1995,Released,6.1,34.0,127.0,"comedy, drama, romance en cheated on, mistreat...",4
4,11862,"baby, midlife crisis, confidence, aging, daugh...",Father of the Bride Part II,Comedy,en,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,United States of America,10/2/1995,Released,5.7,173.0,106.0,comedy en just when george banks has recovered...,5


In [58]:
indices_map = df_merge_ids.set_index("id")
indices_map.head()

,keywords,title,genres,original_language,overview,tagline,production_countries,release_date,status,vote_average,vote_count,runtime,soup,movieId
id,,,,,,,,,,,,,,
862,"jealousy, toy, boy, friendship, friends, rival...",Toy Story,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",,United States of America,30/10/1995,Released,7.7,5415.0,81.0,"animation, comedy, family en led by woody, and...",1
8844,"board game, disappearance, based on children's...",Jumanji,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,United States of America,15/12/1995,Released,6.9,2413.0,104.0,"adventure, fantasy, family en when siblings ju...",2
15602,"fishing, best friend, duringcreditsstinger, ol...",Grumpier Old Men,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,United States of America,22/12/1995,Released,6.5,92.0,101.0,"romance, comedy en a family wedding reignites ...",3
31357,"based on novel, interracial relationship, sing...",Waiting to Exhale,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,United States of America,22/12/1995,Released,6.1,34.0,127.0,"comedy, drama, romance en cheated on, mistreat...",4
11862,"baby, midlife crisis, confidence, aging, daugh...",Father of the Bride Part II,Comedy,en,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,United States of America,10/2/1995,Released,5.7,173.0,106.0,comedy en just when george banks has recovered...,5


In [59]:
def predict_user_rating(userId: int, qualified_movies: pd.DataFrame) -> pd.DataFrame:
    """
    Predict user ratings for qualified movies based on user ID using Singular Value Decomposition (SVD).

    :param userId: The ID of the user.
    :param qualified_movies:  A Pandas DataFrame containing qualified movies data.
    :return: A Pandas DataFrame containing the final qualified movies sorted by estimated user ratings.
    """

    # Calculate estimated user ratings for qualified movies using SVD
    qualified_movies["estimated_user_rating"] = qualified_movies["id"].apply(
        lambda x: svd.predict(userId, indices_map.loc[x]["movieId"]).est
    )
    final_qualified_movies = qualified_movies.sort_values(
        by=["estimated_user_rating"], ascending=False
    )

    return final_qualified_movies

In [60]:
def get_movie_recommendations_hybrid(title: str, userId: int) -> None:
    """
    Get movie recommendations based on a given title and user ID.

    :param title: The title of the movie to find similar movies for.
    :param userId: The ID of the user.
    """
    # Get recommended movie indices based on the given title
    recommended_movie_indices = get_recommended_movie_indices(title)

    # Filter out bad movies and select the top 10 qualified movies
    qualified_movies = filter_out_bad_movies(recommended_movie_indices).head(10)

    # Predict user ratings for qualified movies and select the top recommended movies
    final_qualified_movies = predict_user_rating(userId, qualified_movies).head(5)

    for _, movie in final_qualified_movies.iterrows():
        # Print the movie details
        print(f"Title: {movie['title']}")
        print(f"Overview: {movie['overview']}")
        print(f"Genres: {movie['genres']}")
        print(f"Original Language: {movie['original_language']}")
        print(f"Runtime: {int(movie['runtime'])} mins")
        print(f"Production Countries: {movie['production_countries']}")
        print(f"Release Date: {movie['release_date']}")
        print(
            f"Rating: {movie['vote_average']} out of 10 ({int(movie['vote_count'])} ratings)"
        )
        print("")

In [ ]:
dropdown = widgets.Dropdown(options=df_merge["title"].unique())
search_box = widgets.Text(placeholder="Search movie title...")
userId_text = widgets.Text(placeholder="Enter a user ID")
button = widgets.Button(description="Get Movie Recommendations")
button.layout.width = "200px"

container = widgets.HBox([search_box, userId_text])
container2 = widgets.HBox([dropdown, button])


def on_search_box_value_change(change):
    """
    Event listener for the search box widget. Updates the dropdown options based on the search query.
    """
    search_value = change.new.lower()
    options = df_merge[df_merge["title"].str.lower().str.contains(search_value)][
        "title"
    ].unique()
    dropdown.options = options if len(options) > 0 else ["Movie not found"]
    dropdown.label = options[0] if len(options) > 0 else "Movie not found"


def on_button_click(button):
    """
    Event listener for the button widget. Displays the recommended movies.
    """
    clear_output()
    display(container)
    display(container2)
    try:
        userId = int(userId_text.value)
        title = dropdown.value
        if title != "Movie not found":
            search_box.value = dropdown.value
            get_movie_recommendations_hybrid(title, userId)
        else:
            print("Movie Not Found")

    except:
        print("Please enter a valid user ID!")


# Attach event listeners to the widgets
search_box.observe(on_search_box_value_change, names="value")
button.on_click(on_button_click)

print("Movie Recommendation System")
display(container)
display(container2)

Please enter a valid user ID!
